In [6]:
import pandas as pd
import os
import numpy as np

In [7]:
def pct_change_list_2022(path: str, pi_type: str) -> list[float]:
    df = pd.read_excel(path)
    if pi_type == "consumer":
        col = "Forecast range2 2022"
    elif pi_type == "producer":
        col = "Forecast range1 2022"
    
    df.columns = df.iloc[0]
    df = df[2:].dropna(how="all")
    pct_change = df[col].tolist()[:-5]
    result = []
    for pct in pct_change:
        if pct is np.nan:
            result.append(np.nan)
        elif "to" in pct:
            num = pct.split("to")
            num1, num2 = float(num[0].strip()[:3]), float(num[1].strip()[:3]) 
            result.append(np.mean([num1, num2]))        

    return result  

In [8]:
cpi_path = "./Consumer Price Index/historicalcpi.xlsx"
ppi_path = "./Producer Price Index/historicalppi.xlsx"

def price_index_pct_change(path: str, pi_type: str) -> pd.DataFrame:
    df = pd.read_excel(path)
    df.columns = df.iloc[0]
    if pi_type == "consumer":
        df = df.iloc[1:27, :list(df.columns).index(2021.0) + 1].dropna(how="all")
        pct_change_2022_path = "./Consumer Price Index/CPIforecast.xlsx"
    elif pi_type == "producer":
        df = df.iloc[1:26, :list(df.columns).index(2021.0) + 1].dropna(how="all")
        pct_change_2022_path = "./Producer Price Index/PPIforecast.xlsx"
    cols = [str(year) for year in list(df.columns)]
    cols[1:] = [year[:-2] for year in cols[1:]]
    df.columns = cols

    df.insert(df.shape[1], "2022", pct_change_list_2022(pct_change_2022_path, pi_type))
    df.reset_index(drop=True, inplace=True)
    
    return df 

In [9]:
cpi_path = "./Consumer Price Index/historicalcpi.xlsx"
ppi_path = "./Producer Price Index/historicalppi.xlsx"
paths = [cpi_path, ppi_path]
dirname = "price-index-clean"
os.mkdir(dirname)
for path in paths:
    filename = path.split("/")[1].lower().replace(" ", "-") + "-clean"
    pi_type = filename.split("-")[0]
    df = price_index_pct_change(path, pi_type)
    df.to_csv(f"{dirname}/{filename}.csv", index=False)
    

In [10]:
df = pd.read_excel( "./Consumer Price Index/historicalcpi.xlsx")


,"Annual percent changes in selected Consumer Price Indexes, 1974 through 2021",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,Consumer Price Index item,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All food,14.3,8.5,3.0,6.3,9.9,11.0,8.6,7.8,4.1,...,2.6,1.4,2.4,1.9,0.3,0.9,1.4,1.9,3.4,3.9
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Food away from home,12.7,9.4,6.8,7.6,9.1,11.1,9.9,9.0,5.4,...,2.8,2.1,2.4,2.9,2.6,2.3,2.6,3.1,3.4,4.5
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Food at home,14.9,8.2,2.1,5.9,10.5,10.8,8.1,7.2,3.5,...,2.5,0.9,2.4,1.2,-1.3,-0.2,0.4,0.9,3.5,3.5
7,"Meats, poultry, and fish",2.2,8.5,0.9,-0.6,16.7,14.9,3.7,4.1,4.1,...,3.6,2.1,7.2,1.9,-3.5,-0.1,0.7,1.0,6.3,6.8
8,Meats,1.8,8.5,0.2,-2.3,18.6,17.0,2.9,3.6,4.9,...,3.4,1.2,9.2,3.0,-4.4,-0.6,0.4,1.3,7.4,7.7
9,Beef and veal,2.9,1.0,-3.2,-0.7,22.9,27.4,5.7,0.8,1.4,...,6.4,2.0,12.1,7.2,-6.3,-1.2,1.4,1.6,9.6,9.3
